In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [50]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(r"E:\web_scarping\project_1\chromedriver.exe")
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)

# =====================================================================================
    # get the login icon and click on it 
# =====================================================================================

    # handle cookies
    try:
        cookie = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID, "hs-eu-confirmation-button")))
        if cookie:
            cookie.click()
        else:
            pass
    except Exception as e:
        print("No cookies tab found and the error is {e}")

    try:
        # get the login button
        login = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".nav-right-login a")))
        if login:
            driver.execute_script("arguments[0].scrollIntoView();", login)
            login.click()
        else:
            pass

    except Exception as e:
        print(f"No login tab found and the error is {e}")

# =====================================================================================
    # complete logiin info
# =====================================================================================

    # get the username tab
    try:
        provided_u_name = "haider1805@icloud.com"
        user_name = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID,"user_login")))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")

    # get password tab
    try:
        provided_pass = "Muhssan7865"
        password = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID, "user_pass")))
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}")

    # get the login tab to enter the username and password
    try:
        # login_tab_css = "submit"
        login_tab = driver.find_element(By.XPATH,  "//input[@type='submit' and @value='Login']")
        driver.execute_script("arguments[0].scrollIntoView();", login_tab)
        login_tab.click()
        for i in range(2):
            driver.back()
    except Exception as e:
        print(f"No login tab found and the error is {e}")

# =====================================================================================
    # find the total number of cars
# =====================================================================================
    
    try:
        import time
        time.sleep(1)
        total_num_cars = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ta-total-items-found")))
        if total_num_cars:
            total_cars_list = total_num_cars.text.split(" ")
            total_cars = int(total_cars_list[0])
            print(f"{total_cars} of cars found on this auction")
        else:
            print("No cars found ")
    except Exception as e:
        print(f"No car number card found and error is {e}")
    
# =====================================================================================
    # find the car link and click on it
# =====================================================================================
    try:
        car_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".carousel-item.active img")))
        if car_link:
            driver.execute_script("arguments[0].scrollIntoView();", car_link)
            car_link.click()
            import time
            time.sleep(2)
        else:
            print("No car link found")
    except Exception as e:
        print(f"No car link and error is {e}")

# =====================================================================================
    # for loop till the number of cars
# =====================================================================================
    # to store values
    results = []
    for i in range(total_cars):
        # dict
        details = {}

    # =====================================================================================
        # get basic information
    # =====================================================================================
        try:
            basic_info_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ta-lot-view-bidding")))
            if basic_info_card:
                # car title
                try:
                    car_title = basic_info_card.find_element(By.CSS_SELECTOR, ".ta-lot-title")
                    if car_title:
                        details['Name'] = car_title.text
                    else:
                        details['Name'] = "N/A"
                except Exception as e:
                    print(f"No car title and error is {e}")
                
                # get the lot number
                try:
                    lot_num = basic_info_card.find_element(By.CSS_SELECTOR, ".ta-lot-ID")
                    if lot_num:
                        details['Lot'] = lot_num.text
                    else:
                        details['Lot'] = "N/A"
                except Exception as e:
                    print(f"No lot number and error is {e}")

                # get estimate price
                try:
                    est_price = basic_info_card.find_element(By.CSS_SELECTOR, ".ta-current-price-amount")
                    if est_price:
                        details['Estimate_price'] = est_price.text
                    else:
                        details['Estimate_price'] = "N/A"
                except Exception as e:
                    print(f"No estimate price and error is {e}")

                # get the increment
                try:
                    increment = basic_info_card.find_element(By.CSS_SELECTOR, ".ta-lot-increment-value")
                    if increment:
                        details['Increment'] = increment.text
                    else:
                        details['Increment'] = "N/A"
                except Exception as e:
                    print(f"No increment price and error is {e}")
                
                # get location
                try:
                    location = basic_info_card.find_element(By.CSS_SELECTOR, "div[data-v-2870f41d] strong")
                    if location:
                        details['Location'] = location.text
                    else:
                        details['Location'] = "N/A"
                except Exception as e:
                    print(f"No car location and error is {e}")

                # get tax rates
                try:
                    bayer_prem_label =  basic_info_card.find_element(By.CSS_SELECTOR, "div[data-v-4260925c] label").text
                    bayer_prem_value = basic_info_card.find_element(By.CSS_SELECTOR, "div[data-v-4260925c] span").text

                    bayer_lab = basic_info_card.find_element(By.CSS_SELECTOR, "div[data-v-5be4ada9] label").text
                    bayer_val = basic_info_card.find_element(By.CSS_SELECTOR, "div[data-v-5be4ada9] span").text

                    vat_lab = basic_info_card.find_element(By.CSS_SELECTOR, "div[data-v-796183dc] label").text
                    vat_val = basic_info_card.find_element(By.CSS_SELECTOR, "div[data-v-796183dc] span").text
                    if bayer_prem_label and bayer_prem_value and bayer_lab and bayer_val and vat_lab and vat_val:
                        details[bayer_prem_label] = bayer_prem_value
                        details[bayer_lab] = bayer_val
                        details[vat_lab] = vat_val
                    else:
                        details[bayer_prem_label] = "N/A"
                        details[bayer_lab] = "N/A"
                        details[vat_lab] = "N/A"
                except Exception as e:
                    print(f"No tax rates and error is {e}")
                
                # view information
                try:
                    # view button and information inside it 
                    viewing_button = basic_info_card.find_element(By.XPATH, "//span[contains(text(), 'Viewing Information')]/ancestor::a")
                    if viewing_button:
                        viewing_button.click()
                        time.sleep(1)
                        # get info
                        view_info = driver.find_element(By.CSS_SELECTOR, "div[data-v-210a17f6] p").text
                        if view_info:
                            details['Viewing Information'] = view_info
                            # close the tab
                            close_tab = driver.find_element(By.CSS_SELECTOR, ".close")
                            close_tab.click()
                        else:
                            details['Viewing Information'] = "N/A"
                except Exception as e:
                    print(f"No view information labels and error is {e}")

                # collection information
                try:
                    # view button and information inside it 
                    collection_button = basic_info_card.find_element(By.XPATH, "//span[contains(text(), 'Collection Information')]/ancestor::a")
                    if collection_button:
                        collection_button.click()
                        time.sleep(1)
                        # get info
                        col_info = driver.find_elements(By.CSS_SELECTOR, "div[data-v-210a17f6] p")
                        if col_info:
                            col_info_list= [col.text.strip() for col in col_info]
                            details['Collection Information'] = col_info_list
                            # close the tab
                            col_close_tab = driver.find_element(By.CSS_SELECTOR, ".close")
                            col_close_tab.click()
                        else:
                            details['Collection Information'] = "N/A"
                except Exception as e:
                    print(f"No collection information labels and error is {e}")


            else:
                print("No basic info card")

        except Exception as e:
            print(f"No basic info card and error is {e}") 
        

        # images
        try:
            imgs_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ta-lot-view-images")))
            if imgs_card:
                imgs = imgs_card.find_elements(By.CSS_SELECTOR, ".ta-carousel-thumbnail img")
                if imgs:
                    imgs_list = [img.get_attribute("src") for img in imgs]
                    imgs_str = ", ".join(imgs_list)
                    details['Images'] = imgs_str
                else:
                    details['Images'] = "N/A"
            else:
                print("No image card found")
        except Exception as e:
            print(f"No image card found and error is {e}")

        # get rest of the information
        try:
            items = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.dynamic-field-item")))
            if items:
                for item in items:
                    label = item.find_element(By.CSS_SELECTOR, "h5.dynamic-field-label").text.strip()
                    val = item.find_element(By.CSS_SELECTOR, "div.dynamic-field-value").text.strip()
                    if label and val:
                        details[label] = val
                    else:
                        print("No data found")
            else:
                print("No items found")
        except Exception as e:
            print(f"No description found and error is {e}")
        
        results.append(details)
        
        # get next button
        try:
            next_b = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Next')]")))
            if next_b:
                next_b.click()
            else:
                print("No more cars avaialbe hence the scraper has been completed")
        except Exception as e:
            pass

        


    # time.sleep(5)
    df = pd.DataFrame.from_dict(results)
    df.to_csv("bpi.csv")
    time.sleep(2)
    driver.quit

# provide path according to tyhe auction
path = "https://www.bpiauctions.com/auctions/#/auctions/83696a68-4bba-47ac-9b0b-398e58dff2b1"
scrape(path)


5 of cars found on this auction


In [51]:
df  = pd.read_csv("bpi.csv")
df

,Unnamed: 0,Name,Lot,Estimate_price,Increment,Location,VAT on Buyer's Premium,Buyer's Premium,VAT On Hammer,Viewing Information,Collection Information,Images,Description,Make,Model,Registration,Fuel,Mileage,Transmission,Year / Age
0,0,2023 Volkswagen Transporter T28 Startline Pane...,1,"£11,100",£100,"Worksop, S81",20%,15%,20%,January 22nd between 10am and 3pm,"['Collection Information', 'All Collections ar...",https://media.app.artisio.co/media/dbec1453-ea...,2023 Volkswagen Transporter T28 Startline Pane...,Volkswagen,Transporter T28 Startline,YM23 VEU,Diesel,24855 miles,Manual,2023
1,1,2023 Volkswagen Transporter T28 Startline Pane...,2,"£12,100",£100,"Worksop, S81",20%,15%,20%,January 22nd between 10am and 3pm,"['Collection Information', 'All Collections ar...",https://media.app.artisio.co/media/dbec1453-ea...,2023 Volkswagen Transporter T28 Startline Pane...,Volkswagen,Transporter T28 Startline,YM23 RYB,Diesel,16451 miles,Manual,2023
2,2,2023 Volkswagen Transporter T28 Startline Pane...,3,"£10,000",£100,"Worksop, S81",20%,15%,20%,January 22nd between 10am and 3pm,"['Collection Information', 'All Collections ar...",https://media.app.artisio.co/media/dbec1453-ea...,2023 Volkswagen Transporter T28 Startline Pane...,Volkswagen,Transporter T28 Startline,YM23 RXZ,Diesel,32776 miles,Manual,2023
3,3,2023 Volkswagen Transporter T28 Startline Pane...,4,"£10,000",£100,"Worksop, S81",20%,15%,20%,January 22nd between 10am and 3pm,"['Collection Information', 'All Collections ar...",https://media.app.artisio.co/media/dbec1453-ea...,2023 Volkswagen Transporter T28 Startline Pane...,Volkswagen,Transporter T28 Startline,YO23 LJL,Diesel,34955 miles,Manual,2023
4,4,2023 Volkswagen Transporter T28 Startline Pane...,5,"£11,100",£100,"Worksop, S81",20%,15%,20%,January 22nd between 10am and 3pm,"['Collection Information', 'All Collections ar...",https://media.app.artisio.co/media/dbec1453-ea...,2023 Volkswagen Transporter T28 Startline Pane...,Volkswagen,Transporter T28 Startline,YO23 MKG,Diesel,27655 miles,Manual,2023


In [ ]:
from urllib.parse import urlparse, urljoin

reg_img = df[['Registration', 'Images']]

def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Registration"] # separate reg nums
        image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # loop around the urls of the current row and also save index for further use
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                url = urljoin("https://", url) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(url)

            # check if the parsed url is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                continue
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(url, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Extracts the file extension (e.g., jpg). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")
# Call the function
download_images(reg_img)
